In [1]:

import random
import math
import os
import subprocess

In [2]:
def single_experiment(height: float, dev: float) -> float:
    output = subprocess.check_output(["cargo", "run", "--release", "--", str(height), str(dev)])
    return float(output)

In [19]:
os.chdir("../cgn")

# Function to generate a new solution within given bounds
def generate_new_solution(current, lower_bound, upper_bound, step=1.0):
    return max(lower_bound, min(upper_bound, current + step * (random.random() - 0.5)))

# Simulated Annealing function
def simulated_annealing(optimize_function):
    # Hyperparameters
    initial_temp = 1.0
    final_temp = 0.001
    alpha = 0.9
    max_iterations = 1000

    # Bounds
    height_lower_bound = 100_000
    height_initial = 500_000
    height_upper_bound = 1_000_000
    dev_lower_bound = 1
    dev_initial = 5
    dev_upper_bound = 10

    # Initial solution
    current_height = height_initial
    current_dev = dev_initial
    current_output = optimize_function(current_height, current_dev)

    # Current best solution
    best_height = current_height
    best_dev = current_dev
    best_output = current_output

    temp = initial_temp

    # Annealing process
    while temp > final_temp:
        for i in range(max_iterations):
            # Generate new candidate solution
            new_height = generate_new_solution(current_height, height_lower_bound, height_upper_bound, step=100_000)
            new_dev = generate_new_solution(current_dev, dev_lower_bound, dev_upper_bound, step=1)

            # Evaluate new solution
            new_output = optimize_function(new_height, new_dev)

            # Check if new solution is better or accept it with a certain probability
            if new_output < best_output or random.random() < math.exp((current_output - new_output) / temp):
                current_height = new_height
                current_dev = new_dev
                current_output = new_output

                # Update best solution
                if new_output < best_output:
                    best_height = new_height
                    best_dev = new_dev
                    best_output = new_output

            print(f"\rHeight: {new_height:.4f} Dev: {new_dev:.4f} Output: {new_output:.4f} Temp: {temp:.2f} Prog: {i/max_iterations:.2f} Best: {best_output:.4f}", end="")

        # Cool down
        temp *= alpha

    return best_height, best_dev, best_output

In [20]:
# Example usage
optimal_height, optimal_dev, optimal_output = simulated_annealing(single_experiment)
print("Optimal height:", optimal_height, "Optimal dev:", optimal_dev, "Optimal output:", optimal_output)

Height: 412699.6612 Dev: 9.4558 Output: 4.4891 Temp: 0.66 Prog: 0.96 Best: 4.47800